In [1]:
# Importing required Libraries
import cv2
import os
import numpy as np
import mediapipe as mp

# Load MediaPipe pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Defining a function to load images from the folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

# Folder Paths of Fall and Non-Fall Images
non_fall_images = load_images_from_folder("nonfallyolo")
fall_images = load_images_from_folder("fallyolo")

2024-09-30 19:46:43.206349: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 19:46:43.312724: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 19:46:45.266587: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER:

In [2]:
# Defining function to extract poses
def extract_pose_features(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks is None:
        return None
    # Extracting pose landmarks
    landmarks = np.array([[lmk.x, lmk.y] for lmk in results.pose_landmarks.landmark]).flatten()
    return landmarks

In [3]:
# Initializing empty lists to store features and labels
X = []
y = []

# Extracting features from non-fall images
for image in non_fall_images:
    features = extract_pose_features(image)
    if features is not None:
        X.append(features)
        y.append(0)  # 0 for non-fall

# Extracting features from fall images
for image in fall_images:
    features = extract_pose_features(image)
    if features is not None:
        X.append(features)
        y.append(1)  # 1 for fall

# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

/home/aekankpatel/anaconda3/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [4]:
# Importing necessary libraries for defining model parameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining the neural network model using Keras Sequential API
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),  # Add dropout layer with rate 0.5
    Dense(64, activation='relu'),
    Dropout(0.5),  # Add dropout layer with rate 0.5
    Dense(1, activation='sigmoid')
])

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

/home/aekankpatel/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8086 - loss: 0.5032 - val_accuracy: 0.8756 - val_loss: 0.3646
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8569 - loss: 0.3720 - val_accuracy: 0.8852 - val_loss: 0.3273
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8715 - loss: 0.3431 - val_accuracy: 0.8979 - val_loss: 0.3157
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8826 - loss: 0.3185 - val_accuracy: 0.8939 - val_loss: 0.2867
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8901 - loss: 0.2915 - val_accuracy: 0.9043 - val_loss: 0.2667
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8915 - loss: 0.2834 - val_accuracy: 0.9043 - val_loss: 0.2511
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8975 - loss: 0.2730 - val_accuracy: 0.9091 - val_loss: 0.2485
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8952 - loss: 0.2789 - val_accuracy: 0.

In [5]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Save the model
model.save("Model.h5")

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9040 - loss: 0.2523


Test Loss: 0.23752637207508087
Test Accuracy: 0.9138755798339844
